# Merging BCR & TCR *CITE* and *nonCITE* samples 
## Input
### BCR & TCR *CITE* samples
* Folder path: `./covid_cite_data`
* Each **cite** has a folder; naming pattern: CITEx_BCR/TCR_VHT_cellranger
* **CAUTION**: sample *TP8B* has no corresponding *CITE* sample
* Each **cite** has a `filtered_contig_annotations.csv` file; important columns:
  * **barcode**: each cell has a unique barcode
  * **contig_id**: each contig has a unique contig_id; multiple contigs may belong to one cell
* Each sample also has a `filtered_contig.fasta.fasta` file; search sequence by contig_id

### Demultiplexed BCR & TCR *CITE* samples
* Folder path: `./CITESEQ_demultiplex`
* Each sample **each cite** has a folder; naming pattern: sample name_BCR/TCR_CITEx
* **CAUTION**: sample *TP8B* has no corresponding *CITE* sample
* Each sample has a `filtered_contig_annotations.csv` file; important columns:
  * **barcode**: each cell has a unique barcode
  * **contig_id**: each contig has a unique contig_id; multiple contigs may belong to one cell
* Each sample **NOT** has a `filtered_contig.fasta.fasta` file; so need to search sequence by contig_id in fasta files in the *CITE* folder
    
### BCR & TCR *nonCITE* samples
* Folder path: `./covid_noncite_data`
* Each sample has a folder; naming pattern: sample name_BCR/TCR_VHT_cellranger
* Each sample has a `filtered_contig_annotations.csv` file; important columns:
  * **barcode**: each cell has a unique barcode
  * **contig_id**: each contig has a unique contig_id; multiple contigs may belong to one cell
* Each sample also has a `filtered_contig.fasta.fasta` file; search sequence by contig_id
  
## Output
* Output folder: `./covid_merged_data`
* CAUTION: **ALL** cell **barcodes** together with their **contig ids** in both *csv* and *fasta* are renamed. The new name has a suffix `_nonCITE` or `_CITEx`
* Include 2 files:
  * `filtered_contig_annotations.csv`
  * `filtered_contig.fasta`

## Import packages

In [1]:
from IPython.display import display
import pandas as pd
import os
from Bio import SeqIO
import numpy as np
from shutil import rmtree

## Specify folder

In [2]:
# CITE samples folder
cite_folder = r'./covid_cite_data'
# Demultiplex floder
demultiplex_folder = r'./CITESEQ_demultiplex'
# nonCITE samples folder
noncite_folder = r'./covid_noncite_data'
# output folder
output_folder = r'./covid_merged_data'
if os.path.isdir(output_folder):
    rmtree(output_folder)
os.mkdir(output_folder)

## Define functions to meger data

In [3]:
def saveFile(sample, rename_barcodes, output_name):
    '''save combined csv and fastq files
    sample is a nested dict with 'nonCITE', 'CITEx' as keys
    the item of sample is still a dict with 'file path' and 'seq path' as keys
    '''
    
    to_write_csv = []
    to_write_fastq = []
    
    for k, v in sample.items():
        tmp_df = pd.read_csv(v['file path'])
        # check barcode
        rename_contigs = set()
        for i in tmp_df.index:
            if tmp_df.at[i, 'barcode'] in rename_barcodes:
                # rename barcode
                tmp_df.at[i, 'barcode'] = tmp_df.at[i, 'barcode'] + '_' + k
                # do not forget to rename contig id
                rename_contigs.add(tmp_df.at[i, 'contig_id'])
                tmp_df.at[i, 'contig_id'] = tmp_df.at[i, 'contig_id'] + '_' + k
        to_write_csv.append(tmp_df)
        
        # read fastq
        with open(v['seq path'], 'rt') as seq_file:
            for record in SeqIO.parse(seq_file, 'fasta'):
                if record.id in rename_contigs:
                    record.id = record.id + '_' + k
                    record.description = ''
                    record.name = ''
                    to_write_fastq.append(record)
        
    # save csv file
    combined_df = pd.concat(to_write_csv, ignore_index=True)
    combined_df.to_csv(os.path.join(output_folder, output_name, 'filtered_contig_annotations.csv'), index=False)
    
    # save fastq file
    SeqIO.write(to_write_fastq, os.path.join(output_folder, output_name, 'filtered_contig.fasta'), 'fasta')
        
    return len(set(combined_df['barcode']))

In [4]:
def mergeData(noncite_folder, cite_folder, demultiplex_folder, s_type):
    '''merge BCR or TCR data specified by s_type
    '''
    samples = {}

    # nonCITE samples
    for f in os.listdir(noncite_folder):
        if os.path.isdir(os.path.join(noncite_folder, f)):
            # the folder name has a pattern: sample name_BCR/TCR_VHT_cellranger
            tmp = f.split('_')
            sample_name = tmp[0]
            sample_type = tmp[1]
            
            # only process BCR or TCR data
            if not sample_type == s_type:
                continue
        
            # nested dicts
            if not sample_name in samples:
                samples[sample_name] = {}
        
            samples[sample_name]['nonCITE'] = {'full sample name': sample_name+'_nonCITE',
                                               'file path': os.path.join(noncite_folder, f, 'filtered_contig_annotations.csv'),
                                               'seq path': os.path.join(noncite_folder, f, 'filtered_contig.fasta')}
        
    # Demultiplexed CITE samples
    for f in os.listdir(demultiplex_folder):
        if os.path.isdir(os.path.join(demultiplex_folder, f)):
            # the folder name has a pattern: sample name_BCR/TCR_CITEx
            sample_name, sample_type, cite_run = f.split('_')
            
            # only process BCR or TCR data
            if not sample_type == s_type:
                continue
        
            # nested dicts
            if not sample_name in samples:
                samples[sample_name] = {}
            
            samples[sample_name][cite_run] = {'full sample name': sample_name+'_'+cite_run,
                                               'file path': os.path.join(demultiplex_folder, f, 'filtered_contig_annotations.csv'),
                                               'seq path': os.path.join(cite_folder, cite_run+'_'+sample_type+'_VHT_cellranger', 'filtered_contig.fasta')}
        
    print('Total {} {} sampless'.format(len(samples), s_type))
    
    ## begin merge
    stats = []

    for sample_name, sample in samples.items():
        # sample is still a dict
        stats.append({'sample name': sample_name, 'data sets': len(sample)})
        
        # define output name
        output_name = sample_name + '_' + s_type + '_VHT_cellranger'
        # save data
        if not(os.path.isdir(os.path.join(output_folder, output_name))):
            os.mkdir(os.path.join(output_folder, output_name))
            
        # only 1 data set, no need to merge
        if len(sample) == 1:
            stats[-1]['nCellCITEOne'] = np.nan
            stats[-1]['nCellCITETwo'] = np.nan
            stats[-1]['Overlap2'] = 0
            stats[-1]['Overlap3'] = 0
            stats[-1]['OverlapCITE'] = 0
        
            for k, v in sample.items():
                # save csc file
                data = pd.read_csv(v['file path'])
        
            stats[-1]['nCellNonCITE'] = len(set(data['barcode']))
            stats[-1]['nCellMerged'] = stats[-1]['nCellNonCITE']
        
            # save file, reanme all barcode and contig
            saveFile(sample, set(data['barcode']), output_name) 

        elif len(sample) == 2:
            # 1 CITE data and 1 nonCITE data
            stats[-1]['Overlap3'] = 0
        
            for k, v in sample.items():
                # determine CITE data and nonCITE data
                if k.startswith('CITE'):
                    cite_data = pd.read_csv(v['file path'])
                else:
                    noncite_data = pd.read_csv(v['file path'])
        
            stats[-1]['nCellCITEOne'] = len(set(cite_data['barcode']))
            stats[-1]['nCellCITETwo'] = np.nan
            stats[-1]['nCellNonCITE'] = len(set(noncite_data['barcode']))
            stats[-1]['OverlapCITE'] = 0
        
            # overlap barcodes
            overlap_barcode = set(cite_data['barcode']).intersection(set(noncite_data['barcode']))
            stats[-1]['Overlap2'] = len(overlap_barcode)
        
            # save files, rename all barcodes and contigs
            stats[-1]['nCellMerged'] = saveFile(sample, set(cite_data['barcode']).union(set(noncite_data['barcode'])), output_name)
    
    
        elif len(sample) == 3:
            # 2 CITE data and 1 nonCITE data
            cite_data = []
        
            for k, v in sample.items():
                # determine CITE data and nonCITE data
                if k.startswith('CITE'):
                    cite_data.append(pd.read_csv(v['file path']))
                else:
                    noncite_data = pd.read_csv(v['file path'])
        
            barcode_cite1 = set(cite_data[0]['barcode'])
            barcode_cite2 = set(cite_data[1]['barcode'])
            barcode_noncite = set(noncite_data['barcode'])
        
            stats[-1]['nCellCITEOne'] = len(barcode_cite1)
            stats[-1]['nCellCITETwo'] = len(barcode_cite2)
            stats[-1]['nCellNonCITE'] = len(barcode_noncite)
        
            # check overlap barcode
            overlap3 = barcode_cite1.intersection(barcode_cite2).intersection(barcode_noncite)
            stats[-1]['Overlap3'] = len(overlap3)
            
            # overlap between CITE
            overlapCITE = barcode_cite1.intersection(barcode_cite2)
            stats[-1]['OverlapCITE'] = len(overlapCITE)
            
            overlap2 = barcode_cite1.union(barcode_cite2).intersection(barcode_noncite)
            stats[-1]['Overlap2'] = len(overlap2)
        
            # all barcode need rename
            overlap_barcode =  barcode_cite1.union(barcode_cite2).union(barcode_noncite)
        
            # save files
            stats[-1]['nCellMerged'] = saveFile(sample, overlap_barcode, output_name)

    ## finish merge, show statistics
    stats_df = pd.DataFrame(stats)
    stats_df.set_index('sample name', inplace=True)
    stats_df.loc['Total', :]= stats_df.sum(axis=0)
    stats_df = stats_df[['data sets', 'nCellMerged', 'nCellNonCITE', 'nCellCITEOne', 'nCellCITETwo', 'OverlapCITE', 'Overlap2', 'Overlap3']]
    stats_df.rename(columns={'Overlap2': 'OverlapCITEvsNonCITE', 'Overlap3': 'OverlapIn3Datasets', 'OverlapCITE': 'OverlapBetweenCITE'}, inplace=True)
    print('Summary of merging {} data'.format(s_type))
    display(stats_df)

## Merge BCR data

In [5]:
mergeData(noncite_folder, cite_folder, demultiplex_folder, 'BCR')

Total 10 BCR sampless
Summary of merging BCR data


,data sets,nCellMerged,nCellNonCITE,nCellCITEOne,nCellCITETwo,OverlapBetweenCITE,OverlapCITEvsNonCITE,OverlapIn3Datasets
sample name,,,,,,,,
NS0A,3.0,1326.0,849.0,318.0,159.0,0.0,0.0,0.0
NS0B,3.0,1017.0,813.0,109.0,95.0,0.0,0.0,0.0
NS1A,3.0,734.0,483.0,104.0,147.0,0.0,0.0,0.0
NS1B,3.0,1464.0,1179.0,136.0,149.0,0.0,0.0,0.0
TP8B,1.0,426.0,426.0,NaN,NaN,0.0,0.0,0.0
TP9B,3.0,175.0,134.0,21.0,20.0,0.0,0.0,0.0
TS2A,3.0,727.0,638.0,61.0,28.0,0.0,0.0,0.0
TS2B,3.0,388.0,328.0,26.0,34.0,0.0,0.0,0.0
TS3A,3.0,1241.0,1094.0,60.0,87.0,0.0,0.0,0.0


## Merge TCR data

In [6]:
mergeData(noncite_folder, cite_folder, demultiplex_folder, 'TCR')

Total 10 TCR sampless
Summary of merging TCR data


,data sets,nCellMerged,nCellNonCITE,nCellCITEOne,nCellCITETwo,OverlapBetweenCITE,OverlapCITEvsNonCITE,OverlapIn3Datasets
sample name,,,,,,,,
NS0A,3.0,6970.0,4190.0,1839.0,941.0,2.0,16.0,0.0
NS0B,3.0,1636.0,914.0,375.0,347.0,0.0,1.0,0.0
NS1A,3.0,3314.0,2765.0,276.0,273.0,2.0,2.0,0.0
NS1B,3.0,3673.0,2846.0,392.0,435.0,0.0,6.0,0.0
TP8B,1.0,792.0,792.0,NaN,NaN,0.0,0.0,0.0
TP9B,3.0,92.0,26.0,36.0,30.0,0.0,0.0,0.0
TS2A,3.0,2589.0,1813.0,524.0,252.0,0.0,0.0,0.0
TS2B,3.0,2299.0,1675.0,263.0,361.0,1.0,0.0,0.0
TS3A,3.0,3018.0,2217.0,388.0,413.0,0.0,1.0,0.0
